**Table of Contents**
1. [Recrearea Sesiunii Spark](#recrearea-sesiunii-spark)
1. [Crearea Structurii De Date](#crearea-structurii-de-date)
    1. [Mediul Necesar](#mediul-necesar)
    1. [Procesarea Datelor](#procesarea-datelor)
        1. [Citirea Datelor](#citirea-datelor)
        1. [Parsarea Datelor](#parsarea-datelor)
        1. [Utilizarea Modelelor](#utilizarea-modelelor)
        1. [Scrierea Rezultatelor](#scrierea-rezultatelor)

# Recrearea Sesiunii Spark

Spre deosebire de sesiunile anterioare, vom aduga doua jaruri:
- `org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0`: pentru a ne conecta la **Kafka**
- `postgresql-42.7.3.jar`: pentru a ne conecta la **Postgres**

In [137]:
import numpy as np
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName('Proiect')
    .master('local[*]')
    .config('spark.sql.shuffle.partitions', 48)
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '16g')
    .config('spark.sql.execution.arrow.pyspark.enabled', True)
    .config('spark.executor.memoryOverheadFactor', '0.20')
    .config('spark.dynamicAllocation.enabled', True)
    # kafka
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0')
    # jarul pentru postgres
    .config('spark.jars', '/home/jovyan/.ivy2/jars/postgresql-42.7.3.jar')
    .getOrCreate()
)
spark

In [138]:
import os
KAFKA_BOOTSTRAP_SERVER = 'bd-kafka:29092' # adresa de bootstrap a serverului kafka din Docker
TOPIC = 'bd-proiect' # numele topicului Kafka
sourceArchiveDirOutput=os.path.join(os.getcwd()+"/proiect/output/sourceArchiveDirOutput")

# Crearea Structurii De Date

## Mediul Necesar 

Deoarece pana acum in `LocationTransformer` am folosit group by pe setul de antrenament, iar in streaming putem face group by doar avand un watermark, vom folosi setul de antrenament de pe disk citindu-l normal, nu in streaming, dupa care vom face join cu datele noastre pentru a crea nivelul de risc al locatiilor.

In [139]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml import Transformer
from pyspark.sql import DataFrame
from datetime import datetime
# pt a salva pipelineul
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable, DefaultParamsReader
from pyspark.ml.param import Param, Params


class BaseTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
    """
    O clasa de baza pentru toate transformarile custom pe care le vom face.
    """

    def __init__(self, inputCol=None, outputCol=None):
        super(BaseTransformer, self).__init__()
        self.inputCol = Param(self, "inputCol", "Input column name")
        self.outputCol = Param(self, "outputCol", "Output column name")
        self._setDefault(inputCol=inputCol, outputCol=outputCol)
        self._set(inputCol=inputCol, outputCol=outputCol)

    def getInputCol(self):
        """
        Getter pentru coloana de input.
        """
        return self.getOrDefault(self.inputCol)

    def getOutputCol(self):
        """
        Getter pentru coloana de output.
        """
        return self.getOrDefault(self.outputCol)

    @classmethod
    def read(cls):
        """
        Returneaza un cititor pentru clasa
        """
        return DefaultParamsReader(cls)

    @classmethod
    def load(cls, path):
        """
        Incarca o instanta a clasei dintr-un fisier.
        """
        return cls.read().load(path)

    def save(self, path):
        """
        Salveaza modeulul intr-un fisier shortcut pentru `write().overwrite().save(path)`
        """
        self.write().overwrite().save(path)


split_date_schema = T.StructType([
    T.StructField("month", T.IntegerType(), True),
    T.StructField("day_bucket", T.IntegerType(), True),
    T.StructField("day_of_week", T.IntegerType(), True),
    T.StructField("rounded_hour", T.IntegerType(), True)
])  # structura de date pentru coloanele rezultate


@F.udf(returnType=split_date_schema)
def split_date(date_str: str):
    date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    day = date.day
    day_bucket = 1 if day <= 10 else 2 if day <= 20 else 3
    return {
        "month": date.month,
        "day_bucket": day_bucket,
        "day_of_week": date.weekday(),
        "rounded_hour": date.hour
    }


class DateTransformer(BaseTransformer):
    """
    Un transformer care va separa coloana de date in mai multe coloane: luna, ziua, ziua saptamanii si ora rotunjita.
    """
    # daca flatten este True atunci se va destructura coloana de output
    def __init__(self, inputCol=None, outputCol=None, flatten=False):
        super(DateTransformer, self).__init__(
            inputCol=inputCol, outputCol=outputCol)
        self.flatten = Param(
            self, "flatten", "Whether to flatten the date column")
        self._setDefault(flatten=flatten)
        self._set(flatten=flatten)

    def _transform(self, df: DataFrame):
        if not self.getOutputCol() and not self.getFlatten():
            raise Exception(
                "You must specify an output column or flatten the output.")
        if self.getFlatten() and self.getOutputCol():
            raise Exception(
                "You can't specify an output column and flatten the output at the same time.")

        if self.getFlatten():
            return df.withColumn("temp", split_date(F.col(self.getInputCol())))\
                .select(df.columns + [F.col("temp.*")]).drop("temp")
        return df.withColumn(self.getOutputCol(), split_date(F.col(self.getInputCol())))

    @staticmethod
    def getFlattenCols():
        return ["month", "day_bucket", "day_of_week", "rounded_hour"]

    def getFlatten(self):
        """
        Getter pentru parametrul flatten.
        """
        return self.getOrDefault(self.flatten)


class LocationTransformer(BaseTransformer):
    """
    Un transformer care va imparti locatiile in categorii de risc in functie de procentajul de tranzactii frauduloase.
    """
    def __init__(self, inputCol=None, outputCol=None,train_df_path=os.getcwd()+"/proiect/input/Fraudulent_E-Commerce_Transaction_Data.parquet"):
        super(LocationTransformer, self).__init__(
            inputCol=inputCol, outputCol=outputCol)
        self.precomputed_risk_levels = None
        self.train_df_path=train_df_path


    def _precompute_risk_levels(self):
        """
        Pentru a precomputa nivelurile de risc, vom folosi datele de antrenament de pe disk.
        :return: Un dataframe broadcastat cu nivelurile de risc.
        """
        # Load the training DataFrame (pre-existing historical data)
        train_df = spark.read.parquet(self.train_df_path)
        # Precompute the risk levels based on historical data
        grouped_df = train_df.groupBy(self.getInputCol()).agg(
            F.count('*').alias('total_transactions'),
            F.sum(F.col('Is Fraudulent')).alias('total_fraudulent_transactions')
        ).withColumn(
            'fraud_percentage',
            F.col('total_fraudulent_transactions') / F.col('total_transactions') * 100
        ).withColumn(
            self.getOutputCol(),
            F.when(F.col('fraud_percentage') <= 5, 'Low Risk')
            .when((F.col('fraud_percentage') > 5) & (F.col('fraud_percentage') <= 20), 'Medium Risk')
            .otherwise('High Risk')
        ).select(
            self.getInputCol(),
            self.getOutputCol()
        )

        self.precomputed_risk_levels = F.broadcast(grouped_df)
        
    def _transform(self, df: DataFrame):
        """
        Functia de transformare a transformerului, analoga cu cea de pana acum doar ca facem join cu nivelurile de risc.
        :param df: DataFrame-ul de intrare
        :return:  DataFrame-ul rezultat
        """
        if self.precomputed_risk_levels is None:
            self._precompute_risk_levels()

        return df.join(
            self.precomputed_risk_levels,
            self.getInputCol(),
            'left'
        )
    
class ClassWeightTransformer(BaseTransformer):

    """
    Un transformer care va adauga un weight pentru clasa pozitiva.
    """
    positiveWeight = Param(
        Params._dummy(), "positiveWeight", "Positive weight for the class")  # pentru a putea folosi parametrul in CrossValidator

    def __init__(self, inputCol=None, outputCol=None, positiveWeight=1.0):
        super(ClassWeightTransformer, self).__init__(
            inputCol=inputCol, outputCol=outputCol)
        self._setDefault(positiveWeight=positiveWeight)
        self._set(positiveWeight=positiveWeight)

    def setPositiveWeight(self, value):

        self._set(positiveWeight=value)

    def getPositiveWeight(self):

        return self.getOrDefault(self.positiveWeight)

    def _transform(self, df: DataFrame):

        return df.withColumn(self.getOutputCol(), F.when(F.col(self.getInputCol()) == 1, self.getPositiveWeight()).otherwise(1.0))
    
class LogTransformer(BaseTransformer):
    """
    Un transformer care va aplica logaritmul pe coloana de input.
    """
    def __init__(self, inputCol=None, outputCol=None):
        super(LogTransformer, self).__init__(
            inputCol=inputCol, outputCol=outputCol)

    def _transform(self, df: DataFrame):
        return df.withColumn(self.getOutputCol(), F.log(F.col(self.getInputCol())+1))    

import pyspark.ml.linalg as M
@F.udf(M.VectorUDT())
def convert_to_dense(v):
    if isinstance(v, M.SparseVector):
        return M.Vectors.dense(v.toArray())
    return v

class DenseTransformer(BaseTransformer):
    """
    Un transformer care va converti un SparseVector in DenseVector.
    """
    def __init__(self, inputCol=None, outputCol=None):
        super(DenseTransformer, self).__init__(
            inputCol=inputCol, outputCol=outputCol)

    def _transform(self, df: DataFrame):
        return df.withColumn(self.getOutputCol(), convert_to_dense(F.col(self.getInputCol())))
    
    

Vom citi pipelineurile anterioare pentru a le putea folosi in continuare.

In [140]:
import os
rf_path = os.path.join(os.getcwd()+"/proiect/output/models/whole_train_model")
from  pyspark.ml import PipelineModel

pip_rf = PipelineModel.load(rf_path)

In [141]:
import tensorflow as tf

tf_model=tf.keras.models.load_model(os.path.join(os.path.join(os.getcwd(), 'proiect', 'output', 'tensor', 'model.keras')))
pipe_tf= PipelineModel.load(os.path.join(os.getcwd()+"/proiect/output/tensor/write_pipe_model"))

In [142]:
tf_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 128)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,181 (133.52 KB)

 Trainable params: 11,393 (44.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,788 (89.02 KB)

## Procesarea Datelor 

### Citirea Datelor

In [143]:
df=(
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)
    .option("subscribe", TOPIC)
    .option("startingOffsets", "earliest")
    .load()
)
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Parsarea Datelor

Pentru a avea structura corecta a datelor si coloanele originale de input, vom folosi schema unui fisier _parquet_ de pe disk.

In [144]:
import os
schema=spark.read.parquet(os.path.join(os.getcwd()+"/proiect/output/mini_test_df.parquet")).schema
schema

StructType([StructField('Transaction ID', StringType(), True), StructField('Customer ID', StringType(), True), StructField('Transaction Amount', DoubleType(), True), StructField('Transaction Date', StringType(), True), StructField('Payment Method', StringType(), True), StructField('Product Category', StringType(), True), StructField('Quantity', LongType(), True), StructField('Customer Age', LongType(), True), StructField('Customer Location', StringType(), True), StructField('Device Used', StringType(), True), StructField('IP Address', StringType(), True), StructField('Shipping Address', StringType(), True), StructField('Billing Address', StringType(), True), StructField('Is Fraudulent', LongType(), True), StructField('Account Age Days', LongType(), True), StructField('Transaction Hour', LongType(), True)])

In [145]:
input_cols=[c.name for c in schema.fields]
input_cols

['Transaction ID',
 'Customer ID',
 'Transaction Amount',
 'Transaction Date',
 'Payment Method',
 'Product Category',
 'Quantity',
 'Customer Age',
 'Customer Location',
 'Device Used',
 'IP Address',
 'Shipping Address',
 'Billing Address',
 'Is Fraudulent',
 'Account Age Days',
 'Transaction Hour']

Pentru a putea folosi schema vom converti coloana `value` in _string_, dupa care vom parsa datele cu aceasta din formatul _json_.

In [146]:
df_1=df.withColumn("value", F.from_json(F.col("value").cast("string"), schema))
df_1.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- Transaction ID: string (nullable = true)
 |    |-- Customer ID: string (nullable = true)
 |    |-- Transaction Amount: double (nullable = true)
 |    |-- Transaction Date: string (nullable = true)
 |    |-- Payment Method: string (nullable = true)
 |    |-- Product Category: string (nullable = true)
 |    |-- Quantity: long (nullable = true)
 |    |-- Customer Age: long (nullable = true)
 |    |-- Customer Location: string (nullable = true)
 |    |-- Device Used: string (nullable = true)
 |    |-- IP Address: string (nullable = true)
 |    |-- Shipping Address: string (nullable = true)
 |    |-- Billing Address: string (nullable = true)
 |    |-- Is Fraudulent: long (nullable = true)
 |    |-- Account Age Days: long (nullable = true)
 |    |-- Transaction Hour: long (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-

Cum nu ne trebuie toate coloanele, le vom selecta doar pe cele de interes.

In [147]:
df_2=df_1.select("key","topic","timestamp","value.*")
df_2.printSchema()

root
 |-- key: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- Transaction ID: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Transaction Amount: double (nullable = true)
 |-- Transaction Date: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Customer Age: long (nullable = true)
 |-- Customer Location: string (nullable = true)
 |-- Device Used: string (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- Shipping Address: string (nullable = true)
 |-- Billing Address: string (nullable = true)
 |-- Is Fraudulent: long (nullable = true)
 |-- Account Age Days: long (nullable = true)
 |-- Transaction Hour: long (nullable = true)



### Utilizarea Modelelor

Aplicarea _Random Forest_.

In [148]:
pred_rf=pip_rf.transform(df_2)
pred_rf.printSchema()

root
 |-- Customer Location: string (nullable = true)
 |-- key: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- Transaction ID: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Transaction Amount: double (nullable = true)
 |-- Transaction Date: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Customer Age: long (nullable = true)
 |-- Device Used: string (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- Shipping Address: string (nullable = true)
 |-- Billing Address: string (nullable = true)
 |-- Is Fraudulent: long (nullable = true)
 |-- Account Age Days: long (nullable = true)
 |-- Transaction Hour: long (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_bucket: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- rounded_hour: integer (n

Aplicarea _TensorFlow_.


Distribuirea modelului TensorFlow intampina o problema specifica legata de serializarea cu `pickle`. Modelele TensorFlow pot fi complexe, iar serializarea lor cu `pickle` poate produce erori. Solutia este sa distribuim modelul folosind `broadcast`.

*Pentru a distribui modelul TensorFlow in PySpark, vom urma urmatorii pasi:*

1. **Extragerea greutatilor si structurii modelului**:

    ```python
    tf_model_weights = tf_model.get_weights()
    model_json = tf_model.to_json()
    ```

    - `tf_model_weights` contine greutatile modelului.
    - `model_json` contine structura modelului in format JSON.

1. **Folosirea `broadcast` pentru distribuirea modelului**:

    ```python
    bc_tf_model_weights = spark.sparkContext.broadcast(tf_model_weights)
    bc_tf_model_json = spark.sparkContext.broadcast(model_json)
    ```

    - `spark.sparkContext.broadcast` trimite informatia catre toate nodurile, minimizand traficul de retea prin distribuirea doar o singura data a datelor comune.

1. **Utilizarea modelului pe nodurile worker**:
    - Pe fiecare nod worker, putem accesa greutatile si structura modelului distribuite prin `broadcast`.




In [149]:
# extragerea greutatilor si a structurii modelului
tf_model_weights = tf_model.get_weights()
model_json = tf_model.to_json()

# distribuirea modelului
bc_tf_model_weights = spark.sparkContext.broadcast(tf_model_weights)
bc_tf_model_json = spark.sparkContext.broadcast(model_json)

Pentru a recupera modelul in fieacre _worker_, vom folosi o clasa de tip _singleton_ care va incarca modelul doar o singura data per _worker_.

In [150]:
class ModelManager:
    _model = None

    @staticmethod
    def get_model():
        """
        Functie care returneaza modelul. Daca modelul nu a fost incarcat, il incarca.
        """
        if ModelManager._model is None:
            model = tf.keras.models.model_from_json(bc_tf_model_json.value)
            model.set_weights(bc_tf_model_weights.value)
            ModelManager._model = model
        return ModelManager._model

Pentru a face predictii cu modelul din TensorFlow, vom crea un _UDF_ care va intoarce un output asemantor cu un model ml din pyspark:
- `tf_predictions`: predictia modelului
- `tf_probability`: un vector cu ouputul layerului de iesire al modelului (al functiei sigmoid)

In [151]:
from pyspark.ml.linalg import DenseVector,VectorUDT

def predict_proba(dense_features):
    """
    Functie care returneaza probabilitatea de apartenenta la clasa 1.
    :param dense_features: Un vector dens cu featureurile
    :return: Probabilitatea de apartenenta la clasa 1.
    """
    model = ModelManager.get_model()
    dense_array = np.array(dense_features).reshape(1, -1)
    pred = model.predict(dense_array,verbose=0)
    return pred[0][0]

@F.udf(T.StructType([
    T.StructField("tf_predictions", T.IntegerType(), True),
    T.StructField("tf_probability", VectorUDT(), True)
]))
def predict_tf(dense_features):
    """
    Functie care returneaza predictia si probabilitatea de apartenenta la clasa 1.
    :param dense_features: Un vector dens cu featureurile
    :return: Un struct cu predictia si probabilitatea de apartenenta la clasa 1.
    """
    probability = predict_proba(dense_features)
    prediction = 1 if probability >= 0.5 else 0
    return prediction, DenseVector([probability])

Ca sa incorporam usor modelul TensorFlow in pipeline, vom crea un transformer care va aplica modelul pe datele de intrare, adaugand predictia si probabilitatea de apartenenta la clasa 1.

In [152]:
class TensorFlowTransformer(BaseTransformer):
    """
    Un transformer care va aplica modelul de TensorFlow pe datele de intrare.
    """
    def __init__(self):
        super(TensorFlowTransformer, self).__init__()

    def _transform(self, df: DataFrame):
        return (pipe_tf.transform(df)
                .withColumn("temp", predict_tf(F.col("dense_features"))).select(df.columns + [F.col("temp.*")]).drop("temp"))

In [153]:
# selectam coloanele de intrare
pre_pred_tf=pred_rf.select(input_cols+["key","timestamp","prediction","probability"])
pre_pred_tf.printSchema()

root
 |-- Transaction ID: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Transaction Amount: double (nullable = true)
 |-- Transaction Date: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Customer Age: long (nullable = true)
 |-- Customer Location: string (nullable = true)
 |-- Device Used: string (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- Shipping Address: string (nullable = true)
 |-- Billing Address: string (nullable = true)
 |-- Is Fraudulent: long (nullable = true)
 |-- Account Age Days: long (nullable = true)
 |-- Transaction Hour: long (nullable = true)
 |-- key: binary (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- prediction: double (nullable = false)
 |-- probability: vector (nullable = true)



In [154]:
pred_tf=TensorFlowTransformer().transform(pre_pred_tf)
pred_tf.printSchema()

root
 |-- Transaction ID: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Transaction Amount: double (nullable = true)
 |-- Transaction Date: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Customer Age: long (nullable = true)
 |-- Customer Location: string (nullable = true)
 |-- Device Used: string (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- Shipping Address: string (nullable = true)
 |-- Billing Address: string (nullable = true)
 |-- Is Fraudulent: long (nullable = true)
 |-- Account Age Days: long (nullable = true)
 |-- Transaction Hour: long (nullable = true)
 |-- key: binary (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- prediction: double (nullable = false)
 |-- probability: vector (nullable = true)
 |-- tf_predictions: integer (nullable = true)
 |-- tf_probability: vector (nullable = true)



Pentru a citi coloana `key` in output, o vom converti la tipul `string`.

In [155]:
pred_tf=pred_tf.withColumn("key", F.col("key").cast("string"))
pred_tf.printSchema()

root
 |-- Transaction ID: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Transaction Amount: double (nullable = true)
 |-- Transaction Date: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Customer Age: long (nullable = true)
 |-- Customer Location: string (nullable = true)
 |-- Device Used: string (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- Shipping Address: string (nullable = true)
 |-- Billing Address: string (nullable = true)
 |-- Is Fraudulent: long (nullable = true)
 |-- Account Age Days: long (nullable = true)
 |-- Transaction Hour: long (nullable = true)
 |-- key: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- prediction: double (nullable = false)
 |-- probability: vector (nullable = true)
 |-- tf_predictions: integer (nullable = true)
 |-- tf_probability: vector (nullable = true)



### Scrierea Rezultatelor

Deoarece nu putem scrie in _Postgres_ un vector din _PySpark_, vom converti coloanele `probability` si `tf_probability` in liste, iar pentru aceasta vom folsi _UDF_.

In [156]:

@F.udf(T.ArrayType(T.DoubleType()))
def vector_to_list(v):
    return v.toArray().tolist() if v is not None else None



Pentru a scrie atat in consola cat si in _Postgres_, vom folosi un _foreachBatch_ .

In [157]:
def write_df(batch_df:DataFrame, batch_id:int):
    """
    Functie care scrie un batch in consola si in Postgres.
    :param batch_df: DataFrame-ul batch-ului
    :param batch_id:  Id-ul batch-ului
    """
    print(f"Writing batch {batch_id}")
    batch_df.persist()
    try:
        #scrierea in console
        batch_df.select("key", "timestamp", "prediction", "probability", "tf_predictions", "tf_probability") \
            .write \
            .format("console") \
            .option("truncate", "false") \
            .mode("append")\
            .save()
        print("Data written to console.")
        print("Datele in consola")
        
        #scrierea in postgres
        batch_df.withColumn("probability", vector_to_list(F.col("probability"))) \
            .withColumn("tf_probability", vector_to_list(F.col("tf_probability"))) \
            .write \
            .format("jdbc") \
            .option("url", "jdbc:postgresql://bd-postgres:5432/mydb_proiect") \
            .option("dbtable", "transaction_spark") \
            .option("user", "myuser") \
            .option("password", "mypassword") \
            .option('driver', 'org.postgresql.Driver') \
            .mode("append") \
            .save()
        print("Data written to Postgres.")
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        batch_df.unpersist()

In [158]:
query=(
    pred_tf
    .writeStream
    .foreachBatch(write_df)
    .option("checkpointLocation",sourceArchiveDirOutput)
    .trigger(processingTime="5 seconds")
    .start()
)

query.awaitTermination() 

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [159]:
query.stop()